In [1]:
# ####################################
# Settings & Imports
# ####################################

# Imports from __future__ in case we're running Python 2
from __future__ import division, print_function
from __future__ import absolute_import, unicode_literals

# import my own helper functions
from read import read_sims_result
from clean import cleanup_0IR_exp
from clean import cleanup_0IR_single

# Our numerical workhorses
import numpy as np

# Import pyplot for plotting
import matplotlib.pyplot as plt

# import logistic regression from scikit learn 
from sklearn.linear_model import LogisticRegression

# import model selection stuff from scikit learn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# import feature selection stuff from scikit learn
from sklearn.feature_selection import RFE

# sklearn.metrics
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import brier_score_loss

#
import statsmodels.api as sm

#
# from sklearn import preprocessing

# Import pandas
import pandas as pd

# Magic function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline

# This enables SVG graphics inline.  There is a bug, so uncomment if it works.
# %config InlineBackend.figure_formats = {'svg',}

# This enables high resolution PNGs. SVG is preferred, but has problems
# rendering vertical and horizontal lines
# %config InlineBackend.figure_formats = {'png', 'retina'}

# JB's favorite Seaborn settings for notebooks
rc = {'lines.linewidth': 2, 
      'axes.labelsize': 18, 
      'axes.titlesize': 18, 
      'axes.facecolor': 'DFDFE5'}

# remove some pandas warning
pd.options.mode.chained_assignment = None

Now we want to construct a model to estimate the probability of default based on some parameter when this is no borrowing or lending.

First, let's read and clean up the data.

Then, we observe the data.

In [2]:
# ####################################
# Read input, toggle env variable here
# ####################################
train_on_file = "0622/0IR300s"
df = read_sims_result("/Users/xcheng/Documents/Oberlin/Summer2/DataAnalysis/data/"+train_on_file, 32)
df2 = cleanup_0IR_exp(df, numSim=300, balanced=True, debts=False)

In [39]:
# ####################################
# Make Sure There is NO DEBT in this 0IR simulation
# ####################################
# df3 = df[df["defaults due to negative wealth"] # filter default rows
#   +df["defaults due to deposit shock"]
#   +df['defaults due to interest'] == 0].copy()
# for col in df.columns.values[:31]:
#     print('{0:5s} {1:15f} {2:15f}'.format(col, df3[col].sum(), df[col].sum()))
# print("total debt to pay:", df3["debt to pay"].sum())
# print("total debt owed:", df3["debt owed"].sum())

In [40]:
# ####################################
# look at how things differ for banks with different level of risk aversion
# ####################################
# avg_by_bank = df2.groupby('theta (risk aversion)').mean()
# avg_by_bank[["default-next-wealth", "default-next-deposit"]].plot(
#     kind="line", 
#     figsize=(12,8),
#     title="Frequency & Reason of Default for Each Bank"
# )
# avg_by_bank[["wealth", "deposits", "cash", "assets"]].plot(
#     kind="line", 
#     figsize=(12,8),
#     title="Investment Decisions for Each Bank"
# )
# avg_by_bank[["credit available", "credit issued"]].plot(
#     kind="line", 
#     figsize=(12,8),
#     title="Credit Condition for Each Bank"
# )

In [41]:
# ####################################
# look at the plots for correlation
# ####################################

# to see how independent variables correlate with each other 
# plt.scatter(df2["cash"], df2["credit available"])
# plt.scatter(df2["assets"], df2["credit available"])
# plt.scatter(df2["deposits"], df2["credit available"])
# plt.scatter(df2["cash"], df2["assets"])
# plt.scatter (df2["cash"], df2["deposits"])
# plt.scatter(df2["deposits"], df2["assets"])

# to see how dependent variable relates to independent variables
# plt.scatter(df2["default-next"], df2["cash"])
# plt.scatter(df2["default-next"], df2["assets"])
# plt.scatter(df2["default-next"], df2["deposits"])
# plt.scatter(df2["default-next"], df2["wealth"])

# plot settings
# plt.yscale("symlog")
# plt.xscale("symlog")
# plt.rcParams["figure.figsize"] = [25,15]

In [42]:
# ####################################
# look at how things differ during each period
# ####################################

# Plot time vs default
# df2['period'].hist(by=df2['default-next'], bins=14,  rwidth=0.7, stacked=True)

# Plot time vs independent variables
# avg_by_time = df2.groupby('period').mean()
# avg_by_time[["default-next-wealth", "default-next-deposit"]].plot(
#     kind="line", 
#     figsize=(12,8),
#     title="Frequency & Reason of Default During Each Period"
# )
# avg_by_time[["wealth", "deposits", "cash", "assets"]].plot(
#     kind="line", 
#     figsize=(12,8),
#     title="Investment Decisions During Each Period"
# )
# avg_by_time[["credit available", "credit issued"]].plot(
#     kind="line", 
#     figsize=(12,8),
#     title="Credit Condition During Each Period"
# )

In [43]:
# negative wealth not default
# df[df["wealth"]<0].loc[1:10000,"period":]
# df2[df2["wealth"]<=0]["wealth"]

In [44]:
# come back from 0 wealth
# df2[df2["theta (risk aversion)"]==0.1]

The thing about negative wealth is due to the haircut in asset when calculating wealth.

In [45]:
# check the number of default vs non-default cases
# df2.groupby('default-next').count()

In [46]:
# Try to figure out whether there is Multicollinearity
# df2["cash"]+df2["assets"]*0.8-df2["deposits"]-df2["wealth"]
# np.linalg.eig(X.corr())

In [47]:
# ####################################
# look at the data
# ####################################
# k=1
# sim1 = df.loc[k*15*31:(k+1)*15*31-1]
# sim1_ready = cleanup_0IR_single(sim1, 32)
# sim1_ready[sim1_ready["theta (risk aversion)"] == 0.05]
# sim2 = df.loc[49*15*31:50*15*31-1].copy().reset_index(drop=True)
# sim2_ready = cleanup_0IR_single(sim2, 32)
# sim2_ready
# sim2_ready[sim2_ready["theta (risk aversion)"] == 0.05].loc[:,"period":]
# df2[df2["theta (risk aversion)"] == 0.05].loc[:,"period":]

We want to construst a model to predict default (in the next period) in a simulation where there is no lending or borrowing. 

Variables we can consider includes deposits, cash, assets, credit available, credit issued. 

I exclude theta (risk aversion) because I think it is private info. 

I exclude period because I don't think banks should have this information. It feels like cheating.

I exclude defaults due to interest, debt to pay, debt owed, over leverages because there is no debt. 

I exclude credit issued becuase this should not affect anything when there is no debt. 

I exclude wealth because wealth is a linear combination of deposits, cash and assets.
(this might not be true)

Let's do a logistic regression.

In [3]:
# independent variables (candidates)
independent = ["deposits", "cash", "assets", "credit available", "wealth", "leverage", 
         "dummy-0-leverage",
         "wealth-lag", "deposits-lag", "cash-lag", "assets-lag", "leverage-lag", 
         "credit-available-lag", "credit-issued-lag", "dummy-0-leverage-lag",
         "over-leverage-frequency"]
# dependent variable 0
dependent = "default-next"

In [4]:
X = df2[independent]
y = df2[dependent]

In our data, there are significantly more non-default cases than default cases after I remove cases where the banks default during the previous periods. Over 98% of the cases are non-default. So when we do cross validation, the model we train would just predict not default every time and still get a very high accuracy. 

[DONE] Solution : Throw away lots of non-default cases randomly
    -> get the same amount of default and non-default cases

In [50]:
# correlation matrix
# X.corr()

We sometimes get a warning about complete quasi-separation. What does that mean? Trying to figure out.

Once I remove cash as an independent variable, the warning about possible complete quasi-separation goes away. However, I don't think we should readlly care about complete quasi-separation.

In [51]:
# RFE feature selection
# logreg = LogisticRegression()
# logreg.fit(X,y)
# chosen_rfe_X = []
# for i in range(1,6):
#     rfe = RFE(logreg, i)
#     rfe = rfe.fit(X,y)
#     print(i, "feature(s):", rfe.support_, rfe.ranking_)
#     chosen_rfe_X.append(X.columns.values[rfe.support_])
# chosen_rfe_X.append(X.columns.values)

In [52]:
# fit the model with features selected by RFE
# for predictors in chosen_rfe_X:
#     print(sm.Logit(y, sm.add_constant(df2[predictors])).fit().summary())

In [53]:
# Check the prediction
# m = LogisticRegression()
# m.fit(df2[["cash", "assets"]], df2["default-next"])
# print(m.intercept_, m.coef_)
# print(m.predict([[0,50], [50,0],[50,50],[-100,-100],[1000,1000]]))
# print(m.predict_proba([[10,0.9]]))

In [54]:
# ##############################
# K-fold cross validation
# ##############################
# fold = 9
# kf = KFold(n_splits=fold, shuffle=True)

# for pp in chosen_rfe_X[1:]:
#     xx = df2[pp]
#     score = 0
# #     f1 = 0
#     con = np.array([[0, 0], [0, 0]])
# #     mat = 0
#     brier = 0
#     for train_index, test_index in kf.split(xx):
#         logreg = LogisticRegression()
#         logreg.fit(xx.iloc[train_index], y.iloc[train_index])
#         score += logreg.score(xx.iloc[test_index], y.iloc[test_index])
#         con += confusion_matrix(y.iloc[test_index], logreg.predict(xx.iloc[test_index]))
# #         f1 += f1_score(y.iloc[test_index], logreg.predict(xx.iloc[test_index]))
# #         mat += matthews_corrcoef(y.iloc[test_index], logreg.predict(xx.iloc[test_index]))
#         brier += brier_score_loss(y.iloc[test_index], logreg.predict(xx.iloc[test_index]))
#     print("{}\n {}\n accuracy:{:24}\n  brier:{:24}\n".format(
#         xx.columns.values, con, score/fold, brier/fold))
    

# for p in X.columns.values:
#     xx = df2[p].values.reshape(-1, 1)
#     score = 0
# #     f1 = 0
#     con = np.array([[0, 0], [0, 0]])
#     brier = 0
#     for train_index, test_index in kf.split(xx):
#         logreg = LogisticRegression()
#         logreg.fit(xx[train_index], y[train_index])
#         score += logreg.score(xx[test_index], y[test_index])
#         con += confusion_matrix(y[test_index], logreg.predict(xx[test_index]))
# #         f1 += f1_score(y[test_index], logreg.predict(xx[test_index]))
#         brier += brier_score_loss(y[test_index], logreg.predict(xx[test_index]))
#     print("{}\n {}\n accuracy:{:24}\n brier:{:24}\n".format(
#         p, con, score/fold, brier/fold))

In [55]:
# ##############################
# look at the pattern in defaults across different simulation
# ##############################
# df2[df2["default-next"]==1].loc[:,"sim#"].plot(kind='hist',
#                                                title="# of defaulted banks during each simulation",
#                                                bins=300, 
#                                                figsize=(16, 6)
#                                               )
# df2[df2["default-next"]==1].plot(kind='scatter', 
#                                  x="sim#",
#                                  y="period", 
#                                  figsize=(16,6),
#                                  title="The periods that defaults happen during each simulation"
#                                 )

In [5]:
# ##############################
# Regularization + cross validation 
# trial and error for the best lambda
# ##############################
fold = 12
inv_reg_strength = 0.007
kf = KFold(n_splits=fold, shuffle=True)

accuracy = 0
conf = np.array([[0, 0], [0, 0]])
brier = 0
co_effs = pd.DataFrame(columns = np.append(X.columns.values, "const"), 
                       index=range(fold))
row = 0

for train_index, test_index in kf.split(X):
    lasso = LogisticRegression(penalty="l1", C=inv_reg_strength)
    lasso.fit(X.iloc[train_index], y.iloc[train_index])
    new_row = np.append(lasso.coef_, lasso.intercept_)
    co_effs.loc[row] = new_row
    accuracy += lasso.score(X.iloc[test_index], y.iloc[test_index])
    conf += confusion_matrix(y.iloc[test_index], lasso.predict(X.iloc[test_index]))
    brier += brier_score_loss(y.iloc[test_index], lasso.predict(X.iloc[test_index]))
    row += 1
print("{}\n accuracy:{:24}\n brier:{:24}\n".format(
        conf, accuracy/fold, brier/fold))
co_effs

[[680  77]
 [ 30 727]]
 accuracy:      0.9293161271507729
 brier:     0.07068387284922718



,deposits,cash,assets,credit available,wealth,leverage,dummy-0-leverage,wealth-lag,deposits-lag,cash-lag,assets-lag,leverage-lag,credit-available-lag,credit-issued-lag,dummy-0-leverage-lag,over-leverage-frequency,const
0,0.00423306,-0.00956314,0.0677235,0,-0.242596,0.0217825,0,-0.0473656,0.00187891,-0.00273404,0.00211403,-0.0150683,0,0.0150693,0,0,0
1,0.00663802,-0.0111088,0.0679239,0,-0.254194,0.0181409,0,-0.0403515,0.00122379,-0.00293514,0.000874609,-0.00820271,0,0.0152864,0,0,0
2,0.00400718,-0.00854956,0.0686884,0,-0.249137,0.023178,0,-0.0468782,4.6899e-06,-0.00112782,0.000515429,-0.0153087,0,0.0184226,0,0,0
3,0.00581478,-0.0102271,0.0690069,0,-0.25603,0.0182862,0,-0.0432472,0.00176818,-0.00336732,0.000375856,-0.0101857,0,0.0155308,0,0,0
4,0.00823129,-0.0132155,0.066434,0,-0.248079,0.0169834,0,-0.0489059,0,-0.00224573,0.00438334,-0.00193009,0,0.0151336,0,0,0
5,0.00332374,-0.0100187,0.0501297,-0.00208223,-0.247642,0.0187513,0,-0.0295846,0.00311003,-0.00401617,0.0175406,-0.0101008,0,0.0224954,0,0,0
6,0.0105925,-0.0134412,0.0713725,0,-0.28838,0.0150936,0,-0.026003,1.80495e-06,-0.00257445,0.000339305,-0.00599958,0,0.00861725,0,0,0
7,0.00363999,-0.0082401,0.0730507,0,-0.266356,0.0225915,0,-0.039681,0.000941381,-0.00211013,0.000450693,-0.0152183,0,0.0112107,0,0,0
8,0.00362007,-0.0080264,0.0693677,0,-0.25434,0.0196707,0,-0.044473,6.07265e-06,-0.00150519,0.00284653,-0.011656,0,0.0156891,0,0,0
9,0.00513897,-0.00886062,0.0702316,0,-0.268895,0.016672,0,-0.0468221,1.91453e-06,-0.00269511,0.00715691,-0.00866782,0,0.0123461,0,0,0


In [6]:
# ################################
# Fit the model on all data
# ################################
final = LogisticRegression(penalty="l1", C=inv_reg_strength)
final.fit(X,y)

LogisticRegression(C=0.007, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [7]:
# ################################
# look at the importance of each variable
# importance = coefficient * varaible average
# ################################
var_imp = pd.DataFrame(columns = np.append(X.columns.values, "const"))
coeff = np.append(final.coef_, final.intercept_)
var_imp.loc["coefficient"] = coeff
var_imp.loc["importance"] = coeff * np.append(np.array(X.mean()), 1)
var_imp

,deposits,cash,assets,credit available,wealth,leverage,dummy-0-leverage,wealth-lag,deposits-lag,cash-lag,assets-lag,leverage-lag,credit-available-lag,credit-issued-lag,dummy-0-leverage-lag,over-leverage-frequency,const
coefficient,0.004240,-0.008606,0.069679,0.0,-0.256322,0.020682,0.0,-0.046305,0.000084,-0.001679,0.002778,-0.013659,0.0,0.015441,0.0,0.0,0.0
importance,0.756165,-1.022201,6.202473,0.0,-2.988031,0.233529,0.0,-0.690723,0.012049,-0.183390,0.169971,-0.106240,0.0,0.577356,0.0,0.0,0.0


In [62]:
# ##############################
# Read input for 1 interest rate
# ##############################
df_1 =  read_sims_result("/Users/xcheng/Documents/Oberlin/Summer2/DataAnalysis/data/0625/1IR", 32)
df_1c = cleanup_0IR_exp(df_1, numNode=32, numPeriod=15, numSim=50, balanced=False)

In [64]:
# ##############################
# Examine the prediction interest rate
# ##############################
print(final.score(df_1c[independent], df_1c[dependent]))
print(confusion_matrix(df_1c[dependent], final.predict(df_1c[independent])))
print(brier_score_loss(df_1c[dependent], final.predict(df_1c[independent])))

In [65]:
# ##############################
# Read input for 2 interest rates
# ##############################
df_2 =  read_sims_result("/Users/xcheng/Documents/Oberlin/Summer2/DataAnalysis/data/0625/2IR", 32)
df_2c = cleanup_0IR_exp(df_2, numNode=32, numPeriod=15, numSim=50, balanced=False)

In [66]:
# ##############################
# Examine the prediction for 2 interest rates
# ##############################
print(final.score(df_2c[independent], df_2c[dependent]))
print(confusion_matrix(df_2c[dependent], final.predict(df_2c[independent])))
print(brier_score_loss(df_2c[dependent], final.predict(df_2c[independent])))

As we can see, the prediction is fairly good.

We have a lot of false negatives, but hopefully the network part would help with this.

An interesting observation: the more sample we have in 0IR, the more false negatives, the less true negatives we get (less accurate predictions).